In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
TRN_cat = pd.HDFStore('/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/TRN_navi_catalog.h5')
TRN_cat_df = TRN_cat['df']
TRN_cat_df.head()

,Angle1,Angle2,Angle3,des1,des2,des3,lon1,lat1,r1,lon2,lat2,r2,lon3,lat3,r3
0,38.505996,17.784368,123.709636,2.061152,2.063950,7.891946,-65.477810,-44.911663,17.598053,-66.812988,-44.115738,6.351422,-67.501326,-44.443916,3.477863
1,123.709636,17.784368,38.505996,2.061152,7.891946,2.063950,-65.477810,-44.911663,17.598053,-67.501326,-44.443916,3.477863,-66.812988,-44.115738,6.351422
2,38.505996,123.709636,17.784368,2.063950,2.061152,7.891946,-66.812988,-44.115738,6.351422,-65.477810,-44.911663,17.598053,-67.501326,-44.443916,3.477863
3,17.784368,123.709636,38.505996,2.063950,7.891946,2.061152,-66.812988,-44.115738,6.351422,-67.501326,-44.443916,3.477863,-65.477810,-44.911663,17.598053
4,123.709636,38.505996,17.784368,7.891946,2.061152,2.063950,-67.501326,-44.443916,3.477863,-65.477810,-44.911663,17.598053,-66.812988,-44.115738,6.351422


In [3]:
# LOAD ALL IMAGES:
dt = 10
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [4]:
def verify(save, tol = 3):
    c1_t1 = [save[0].x1, save[0].y1, save[0].r1]
    c2_t1 = [save[0].x2, save[0].y2, save[0].r3]
    c3_t1 = [save[0].x3, save[0].y2, save[0].r3]

    c1_t2 = [save[1].x1, save[1].y1, save[1].r1]
    c2_t2 = [save[1].x2, save[1].y2, save[1].r3]
    c3_t2 = [save[1].x3, save[1].y2, save[1].r3]

    centroid_1 = [(c1_t1[0]+c2_t1[0]+c3_t1[0])/3, (c1_t1[1]+c2_t1[1]+c3_t1[1])/3]
    centroid_2 = [(c1_t2[0]+c2_t2[0]+c3_t2[0])/3, (c1_t2[1]+c2_t2[1]+c3_t2[1])/3]

    delta_x = abs(centroid_1[0]-centroid_2[0])
    delta_y = abs(centroid_1[1]-centroid_2[1])

    if c1_t1[2]-c1_t2[2] < tol: # Se il primo cratere è il primo cratere bis
        if (abs(abs(c1_t1[0]-c1_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c1_t2[1])-delta_y) < tol):
            return True
    elif c1_t1[2]-c2_t2[2] < tol: # primo - secondo
        if (abs(abs(c1_t1[0]-c2_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c2_t2[1])-delta_y) < tol):
            return True
    elif c1_t1[2]-c2_t2[2] < tol: # primo - terzo
        if (abs(abs(c1_t1[0]-c3_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c3_t2[1])-delta_y) < tol):
            return True
    else: return False

# ODOMETRY

In [ ]:
############################################
dt = 10
############################################
df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)
x, y, z = real_X[0], real_Y[0], real_Z[0]

In [11]:
idx = 5
tol = 5
# IMG1:
img1=cv2.imread(dict[str(idx+1)])
craters_det1 = detect(img1)
craters_cat1 = crater_catalogued([x,y,z]) # OUTPUTS: x,y, radius (px)

# IMG2:
img2=cv2.imread(dict[str(idx+2)])
craters_det2 = detect(img2)

# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
df_craters_det2 = sort_mat(craters_det2)
# Find all triplets:
H1 = find_all_triplets(craters_det1)
H2 = find_all_triplets(craters_det2)
QUERY1= pd.DataFrame(H1, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
QUERY2= pd.DataFrame(H2, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
# remove wrong triplet:
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 =QUERY2[ (QUERY2.Angle1 > 10) & (QUERY2.Angle2 > 10) & (QUERY2.Angle3 > 10) ].reset_index(drop=True)
##########################################
# SEARCH
##########################################
found = False
for idx in range(QUERY1.shape[0]):
    Angle1_Q1 = QUERY1.Angle1[idx]
    Angle2_Q1 = QUERY1.Angle2[idx]
    Angle3_Q1 = QUERY1.Angle3[idx]
    d1_Q1 = QUERY1.des1[idx]
    d2_Q1 = QUERY1.des2[idx]
    d3_Q1 = QUERY1.des3[idx]

    SEARCH = QUERY2[ ((abs(QUERY2.Angle1 - Angle1_Q1) < tol) & (abs(QUERY2.Angle2 - Angle2_Q1) < tol) & (abs(QUERY2.Angle3 - Angle3_Q1) < tol))   \
                      |  ((abs(QUERY2.Angle1 - Angle2_Q1) < tol) & (abs(QUERY2.Angle2 - Angle3_Q1) < tol) & (abs(QUERY2.Angle3 - Angle1_Q1) < tol)) \
                      |  ((abs(QUERY2.Angle1 - Angle3_Q1) < tol) & (abs(QUERY2.Angle2 - Angle1_Q1) < tol) & (abs(QUERY2.Angle3 - Angle2_Q1) < tol))]

    SEARCH = SEARCH  [ ((abs(QUERY2.des1- d1_Q1) < tol) & (abs(QUERY2.des2- d2_Q1) < tol) & (abs(QUERY2.des3- d3_Q1) < tol)) \
                     | ((abs(QUERY2.des1- d2_Q1) < tol) & (abs(QUERY2.des2- d3_Q1) < tol) & (abs(QUERY2.des3- d1_Q1) < tol)) \
                     | ((abs(QUERY2.des1- d3_Q1) < tol) & (abs(QUERY2.des2- d1_Q1) < tol) & (abs(QUERY2.des3- d2_Q1) < tol)) ]
    if SEARCH.shape[0] != 0:
        print('Maybe I found!')
        save = [QUERY1.iloc[idx], SEARCH.iloc[0]]
        switch = verify(save, tol = 10)
        if switch:
            print('FOUND!')
            found = True
            break
        else:
            continue
if found:
    a = save[0]
    b = save[1]
    crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
    crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

    plt.figure(dpi=200)
    plt.subplot(121)
    cp1 = img1.copy()
    IMG1 =  img_plus_crts(cp1, crts1, color="red")
    plt.imshow(IMG1)

    plt.subplot(122)
    cp2 = img2.copy()
    IMG2 =  img_plus_crts(cp2, crts2, color="red")
    plt.imshow(IMG2)
    plt.show()
else:
    print("Match can't find any triplets.")

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Maybe I found!
FOUND!


# CATALOG NAVI

In [30]:
def verifica(SEARCH, filename):
    t = filename.split('_')[-1].split('.')
    longy = t[0]+'.'+t[1]
    longy = float(longy)    
    for triplet in SEARCH.iloc:

            cond1 = (triplet.lat1 < 2) & (triplet.lat1 > -2) & (triplet.lon1 < longy+2) & (triplet.lon1 > longy-2)
            cond2 = (triplet.lat2 < 2) & (triplet.lat2 > -2) & (triplet.lon2 < longy+2) & (triplet.lon2 > longy-2)
            cond3 = (triplet.lat3 < 2) & (triplet.lat3 > -2) & (triplet.lon3 < longy+2) & (triplet.lon2 > longy-2)

            if (cond1) & (cond2) & (cond3):
                return True
            else: return False


def print_res(SEARCH, filename):
    t = filename.split('_')[-1].split('.')
    longy = t[0]+'.'+t[1]
    longy = float(longy)    
    for triplet in SEARCH.iloc:
            if (triplet.lat1 < 2) & (triplet.lat1 > -2) & (triplet.lon1 < longy+3) & (triplet.lon1 > longy-3):
                print(triplet)

In [27]:
idx = 0
# IMG1:
filename = dict[str(idx+1)]
img1=cv2.imread(filename)
craters_det1 = detect(img1)

# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
# Find all triplets:
H1 = find_all_triplets(craters_det1)
QUERY1= pd.DataFrame(H1, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
QUERY2 = TRN_cat_df
# remove wrong triplet:
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)

#SEARCH
tol = 5

QUERY1_arr = np.array(QUERY1)
for i in range(QUERY1_arr.shape[0]):
    row = QUERY1_arr[i]
    printProgressBar(i+1,QUERY1.shape[0], printEnd='')

    Angle1_Q1, Angle2_Q1, Angle3_Q1 = row[0],row[1],row[2] 
    d1_Q1, d2_Q1, d3_Q1 = row[3], row[4], row[5] 
    SEARCH = QUERY2[ (((abs(QUERY2.Angle1 - Angle1_Q1) < tol) & (abs(QUERY2.Angle2 - Angle2_Q1) < tol) & (abs(QUERY2.Angle3 - Angle3_Q1) < tol))         \
                         |  ((abs(QUERY2.Angle1 - Angle2_Q1) < tol) & (abs(QUERY2.Angle2 - Angle3_Q1) < tol) & (abs(QUERY2.Angle3 - Angle1_Q1) < tol))   \
                         |  ((abs(QUERY2.Angle1 - Angle3_Q1) < tol) & (abs(QUERY2.Angle2 - Angle1_Q1) < tol) & (abs(QUERY2.Angle3 - Angle2_Q1) < tol)))  \
        &                   (((abs(QUERY2.des1- d1_Q1) < tol) & (abs(QUERY2.des2- d2_Q1) < tol) & (abs(QUERY2.des3- d3_Q1) < tol))                       \
                         |  ((abs(QUERY2.des1- d2_Q1) < tol) & (abs(QUERY2.des2- d3_Q1) < tol) & (abs(QUERY2.des3- d1_Q1) < tol))                        \
                         |  ((abs(QUERY2.des1- d3_Q1) < tol) & (abs(QUERY2.des2- d1_Q1) < tol) & (abs(QUERY2.des3- d2_Q1) < tol))) ]


    if SEARCH.shape[0] != 0:
        saved_idx = idx
        if verifica(SEARCH, filename):
            print_res(SEARCH, filename)
            break

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |█████████████████████████---------------------------------------------------------------------------| 25.6% 

KeyboardInterrupt: 

In [ ]:
SEARCH

In [ ]:
if verifica(SEARCH, filename):
    F = QUERY1.iloc[saved_idx]
    triplet_fonte = [F.x1,F.y1, F.r1, F.x2,F.y2,F.r2, F.x3,F.y3,F.r3]
    cr1 = triplet_fonte[0:3]
    cr2 = triplet_fonte[3:6]
    cr3 = triplet_fonte[6:9]
    triplet_fonte = np.vstack([cr1,cr2,cr3])

    min_lon = np.argmin([cr1[0], cr2[0], cr3[0]  ])
    left = triplet_fonte[min_lon]

    max_lon = np.argmax([cr1[0], cr2[0], cr3[0]  ])
    right = triplet_fonte[max_lon]
    #########################################################################################
    ################          LOOP THROUGH SEARCH            ################################
    #########################################################################################

    for k in range(SEARCH.shape[0]):
        hp = SEARCH.iloc[k]
        triplet_hp = [hp.lon1,hp.lat1, hp.r1, hp.lon2,hp.lat2,hp.r2, hp.lon3,hp.lat3,hp.r3]


        cr1_h = triplet_hp[0:3]
        cr2_h = triplet_hp[3:6]
        cr3_h = triplet_hp[6:9]
        triplet_hp = np.vstack([cr1_h,cr2_h,cr3_h])

        min_lon_h = np.argmin([cr1_h[0], cr2_h[0], cr3_h[0]  ])
        left_h = triplet_hp[min_lon]

        max_lon_h = np.argmax([cr1[0], cr2[0], cr3[0]  ])
        right_h   = triplet_hp[max_lon]

        l_ratio = left[2]/left_h[2]
        r_ratio = right[2]/right_h[2]

        if abs(l_ratio-r_ratio) < 0.3:
            print('OK')
            break
        else:
            print('NO')
    #####################################################################################
    # Relative on board:
    x1, y1, r1 = cr1[0], cr1[1], cr1[2]
    x2, y2, r2 = cr2[0], cr2[1], cr2[2]
    x3, y3, r3 = cr3[0], cr3[1], cr3[2]

    C = np.zeros(2)  # centroid relative
    C[0] = (x1+x2+x3)/3
    C[1] = (y1+y2+y3)/3

    CAMx, CAMy = 850/2, 850/2
    CAM_cen = [CAMx,CAMy]
    ####################################################################################
    # Absolute on DEM:
    x1_h, y1_h, r1_h = cr1_h[0], cr1_h[1], cr1_h[2]
    x2_h, y2_h, r2_h = cr2_h[0], cr2_h[1], cr2_h[2]
    x3_h, y3_h, r3_h = cr3_h[0], cr3_h[1], cr3_h[2]

    C_h = np.zeros(2)  # centroid absolute
    C_h[0] = (x1_h+x2_h+x3_h)/3
    C_h[1] = (y1_h+y2_h+y3_h)/3


    delta = C-CAM_cen
    px2km = 1/np.mean([l_ratio, r_ratio])
    delta_in_km = delta*px2km
    km2deg = 1/deg2km
    delta_in_deg = delta_in_km*km2deg
    Pos_LL = C_h - delta_in_deg
    d = 850*px2km
    FOV_deg = 90
    FOV_rad = np.deg2rad(FOV_deg)
    Altitude = d/(2*np.tan(FOV_rad/2))
    Pos_LLH = np.hstack([Pos_LL, Altitude])  
    x,y,z = spherical2cartesian(Pos_LLH[2],Pos_LLH[1],Pos_LLH[0])
    print(Pos_LLH)

In [ ]:
# from copy import deepcopy

# def super_search(q1,q2, filename, tol):

#     res1 = deepcopy(q1)
#     res2 = deepcopy(q2)

#     def AngleChanger(AngleName,tol, q1=q1):
#         q1[AngleName] += tol
#         return q1

#     def reset(q1=q1,q2=q1, re1=res1, re2=res2):
#         q1 = re1
#         q2 = re2
#         return q1, q2

# # CONTINUE WORKING...

#     r = try_search(q1,q2)
#     if r is not None: return r

#     # Inner Loop:
#     q1 = AngleChanger('Angle1',tol)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle2',tol)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle2',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle2',1)
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle2',1)
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle2',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle2',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle2',-1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle2',-1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     # NEGATIVES:
#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle2',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r


#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle2',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle2',1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle2',-1)
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle2',1)
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r
    
#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle2',-1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle2',1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r
    
#     q1,q2 = reset()
#     q1 = AngleChanger('Angle2',-1)
#     q1 = AngleChanger('Angle3',1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle2',1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     if r is not None: return r

#     q1,q2 = reset()
#     q1 = AngleChanger('Angle1',-1)
#     q1 = AngleChanger('Angle2',-1)
#     q1 = AngleChanger('Angle3',-1)
#     r = try_search(q1,q2)
#     q1,q2 = reset()
#     if r is not None: return r
# ############ ENDING
#     return None




# def try_search(q1,q2, filename=filename):

#     S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
#     if verifica(S, filename): return S

#     q1=swap_df_columns('Angle1','Angle2',q1)
#     S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
#     if verifica(S, filename): return S

#     q1=swap_df_columns('Angle2','Angle1',q1) #reverse
#     q1=swap_df_columns('Angle1','Angle3',q1)
#     S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
#     if verifica(S, filename): return S

#     q1=swap_df_columns('Angle3','Angle1',q1) #reverse
#     q1=swap_df_columns('Angle2','Angle3',q1)
#     S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
#     if verifica(S, filename): return S

#     q1=swap_df_columns('Angle3','Angle2',q1) #reverse
#     q1=swap_df_columns('Angle1','Angle3',q1)
#     q1=swap_df_columns('Angle2','Angle3',q1)
#     S = pd.merge(q1,q2, on=['Angle1','Angle2','Angle3'],   how='inner', indicator=False)
#     if verifica(S, filename): return S

#     return None

In [ ]:
# # Searching:
# count,d, found = 0, 10, False
# # for i in QUERY1.iloc:
# for idx in range(QUERY1.shape[0]):
#     if found:
#         break

#     i = QUERY1.iloc[idx]
#     count+=1
#     printProgressBar(count, QUERY1.shape[0], printEnd='')
    
#     for idy in range(QUERY2.shape[0]):
#         j = QUERY2.iloc[idy]
#         a1,a2,a3 = i.Angle1,i.Angle2,i.Angle3
#         b1,b2,b3 = j.Angle1,j.Angle2,j.Angle3
#         x1,x2,x3 = i.r1,i.r2,i.r3
#         y1,y2,y3 = j.r1,j.r2,j.r3
# #       switch
#         d1 = abs(a1-b1)+abs(a2-b2)+abs(a3-b3)
#         d2 = abs(a1-b2)+abs(a2-b3)+abs(a3-b1)
#         d3 = abs(a1-b3)+abs(a2-b1)+abs(a3-b2)
# #       switch
#         r1 = abs(x1-y1)+abs(x2-y2)+abs(x3-y3)
#         r2 = abs(x1-y2)+abs(x2-y3)+abs(x3-y1)
#         r3 = abs(x1-y3)+abs(x2-y1)+abs(x3-y2)

#         d = min(d1,d2,d3)
#         r = min(r1,r2,r3)
#         if (d < 2) & (r < 5):
#             print('\nFOUNDED')
#             save = [idx, idy]
#             print(save)
#             print(d,r)
#             found = True
#             break
# print('\n')
# if found:
#     a = QUERY1.iloc[save[0]]
#     b = QUERY2.iloc[save[1]]
#     crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
#     crts2 = np.vstack([[b.x1,b.y1,b.r1],[b.x2,b.y2,b.r2],[b.x3,b.y3,b.r3]])

#     plt.figure(dpi=200)
#     plt.subplot(121)
#     cp1 = img1.copy()
#     IMG1 =  img_plus_crts(cp1, crts1, color="red")
#     plt.imshow(IMG1)

#     plt.subplot(122)
#     cp2 = img2.copy()
#     IMG2 =  img_plus_crts(cp2, crts2, color="red")
#     plt.imshow(IMG2)
#     plt.show()
# else:
#     print("Match can't find any triplets.")

In [ ]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [ ]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [ ]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [ ]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [ ]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)